In [1]:
import ray
import pandas as pd
import pickle
import genetic_algorithm as ga

### tuning parameters

In [1]:
population_size_list = [20,50]

# crossover parameters
crossover_probability_list = [0.6, 1.0]
crossover_type_list = [1, 2] 
fitness_function_list = ['cost', 'distance']

# selection parameters
elitism_ratio_list = [0.2, 0.4] # must be even!
max_parent_allowance_list = [0.125, 0.40]

#mutation parameters
mutation_probability_list = [0.1, 0.5] 
mutation_rate_list = [0.1, 0.5] 

# algorithm settings
max_iter = 300 
k = 10 
stop_thrs = 10
mutation_level = 0.2

run_num = 3
individual_size = 120

In [ ]:
from itertools import permutations  
name = "Adam"
   
result = permutations(name)  
  
for j in list(result):  
    print(j)

In [3]:
params_pool = []

for population_size in population_size_list:
    for fitness_function in fitness_function_list:
        for crossover_type in crossover_type_list: 
            for crossover_probability in crossover_probability_list:
                for elitism_ratio in elitism_ratio_list:
                    for max_parent_allowance in max_parent_allowance_list:
                        for mutation_probability in mutation_probability_list:
                            for mutation_rate in mutation_rate_list:
                                for run in range(1, run_num+1):

                                    parameters = [
                                    run, population_size, k,\
                                    fitness_function,  crossover_type, crossover_probability, \
                                    elitism_ratio, max_parent_allowance, \
                                    mutation_probability, mutation_rate, max_iter, individual_size,\
                                    stop_thrs, mutation_level \
                                    ]

                                    params_pool.append(parameters)

### parallel processing with ray

In [10]:
ray.init()

2021-01-24 12:50:45,076	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.11',
 'raylet_ip_address': '192.168.0.11',
 'redis_address': '192.168.0.11:6379',
 'object_store_address': 'tcp://127.0.0.1:57777',
 'raylet_socket_name': 'tcp://127.0.0.1:62297',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\ISMAIL~1\\AppData\\Local\\Temp\\ray\\session_2021-01-24_12-50-43_928425_21336',
 'metrics_export_port': 59763,
 'node_id': 'c069313bce487637aa47df319d1f8744890b00f4'}

In [11]:
@ray.remote
def run(parameters):
    global idx
    idx += 1 
    try:
        result = ga.run(parameters)
    except:
        print('An error occuered with ', parameters)
        return (parameters, {})
    print('run',idx, parameters, ' - ', "{:.2f}".format(result['run_time']) , ' dk... ')
    return (parameters, result)

In [ ]:
futures = [run.remote(params) for params in params_pool_piece]
results = ray.get(futures)

(pid=29228) run 1 ['120_2', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.31  dk... 
(pid=25876) run 1 ['120_1', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.34  dk... 
(pid=23004) run 1 ['120_2', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.40  dk... 
(pid=21084) run 1 ['120_1', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.45  dk... 
(pid=25032) run 1 ['120_2', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.47  dk... 
(pid=28896) run 1 ['120_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.49  dk... 
(pid=14708) run 1 ['120_2', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.1, 300, 120, 10, 0.2]  -  3.94  dk... 
(pid=13812) run 1 ['120_1', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.1, 300, 120, 10, 0.2]  -  3.97  dk... 
(pid=5436) run 1 ['120_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.1, 300, 120, 10, 0.2]  

(pid=23056) run 3 ['120_2', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.4, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.30  dk... 
(pid=29228) run 3 ['120_2', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.4, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.30  dk... 
(pid=15540) run 3 ['120_2', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.4, 0.1, 0.5, 300, 120, 10, 0.2]  -  3.44  dk... 
(pid=11400) run 2 ['360_2', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.5, 300, 360, 10, 0.2]  -  9.50  dk... 
(pid=23292) run 2 ['360_1', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.5, 300, 360, 10, 0.2]  -  9.77  dk... 
(pid=32660) run 3 ['360_1', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.4, 0.1, 0.1, 300, 360, 10, 0.2]  -  8.27  dk... 
(pid=24408) run 2 ['360_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.4, 0.1, 0.1, 300, 360, 10, 0.2]  -  8.42  dk... 
(pid=9500) run 2 ['360_2', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.5, 300, 360, 10, 0.2]  -  10.08  dk... 
(pid=32688) run 2 ['360_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.125, 0.5, 0.5, 300, 360, 10, 0.2]  -  10.21 

(pid=10316) run 5 ['120_2', 3, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  20.08  dk... 
(pid=9704) run 4 ['120_1', 1, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.1, 300, 120, 10, 0.2]  -  24.38  dk... 
(pid=23004) run 5 ['120_2', 1, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.1, 300, 120, 10, 0.2]  -  25.06  dk... 
(pid=30152) run 5 ['120_1', 3, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.1, 300, 120, 10, 0.2]  -  25.51  dk... 
(pid=5436) run 6 ['120_1', 2, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  22.04  dk... 
(pid=14708) run 6 ['120_1', 1, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  22.11  dk... 
(pid=25876) run 5 ['120_1', 3, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  23.36  dk... 
(pid=9500) run 4 ['120_2', 2, 50, 10, 'distance', 1, 0.6, 0.2, 0.125, 0.1, 0.5, 300, 120, 10, 0.2]  -  23.19  dk... 
(pid=21332) run 6 ['240_1', 1, 50, 10, 'distance', 1, 0.6, 

In [ ]:
res_file_name = 'results.pickle'

col_names = ['#','Dataset', 'Run', 'Pop Size', 'K', 'Fitness Func', 'Crossover Type', 'Crossover Prob', 'Elitism Ratio', \
            'Max Parent Allow', 'Mutation Prob', 'Mutation Rate', 'Max Iter', 'Individual Size','Solution Costs', 'Best Solution', \
            'Stopping Cond', 'Run Time','Min Cost']

res_df = pd.DataFrame(columns=col_names)

### results

In [ ]:
index = len(res_df)

for params, res in results:
    
    if len(res)==0:

        solns = [index] + params[0:13] + ['']*5
    else:    
        solns = [index] + params[0:13] + [res['cost_func'], res['best_soln'], \
                                      res['stop_cond'], res['run_time'], res['min_cost']]
        
    res_df = res_df.append(pd.DataFrame([solns], columns=col_names),ignore_index=True)
    
    index +=1
    
res_df.to_pickle(res_file_name)